In [1]:
import gurobipy as gb
from gurobipy import GRB
from itertools import chain
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dropbox
from io import BytesIO

Get access to dropbox dataset

In [2]:
ACCESS_TOKEN = "sl.u.AGKsSIc9iNXkBphnfqZto2jbibfqFtGeeK4XGE3WYws-lytdx8e8XQa2HK8Iv7L0WGatqstpX8jBB_25o92tyofdnGDMd16QBY6WZrRFSTVMqhLQrijYIVUdkVOIJuD5m_v93ctgnFq2d7g7GvygWIFFWyoENIZ1yag3eXhRurJdZGX0RDwpmauyJQDBlBkhAu_2ZNOn8FMIi2O2L8Jq7e7PxLHXTfN3qPIElEJP0wkjCukt0l96QGD2M4pow0P8Y9WIXZHrMVFLE0RSErvJ-UdSXV2g9-GpRFC2vlDhFGZ2ZVUNawsF9L7YdhEkAz5hXC1KHiV892LFMAEOImiqlfX4VEijWmW7-aGEl-wJsbciXUexXJrgahEfSzWpLEvgdtR9flfPTPzQG-oOhaDFnF-2MGjq4pwZOobUHE7KEDCwdI508e5DOkS0HlD4C87TfS-Z62F2rePnN6g4sRXyWWzac0yUAptx6sr-K2vf0jsmLMy6PCQCB89sfgqqqETuNMoJDocPFsMq6rAwgZj5INYP52yGHAl9YP-a7dKKnvuIZ8o_-sEVs1wEh5XnrJFukAdy7fPUZ1X4dchuX86yD7Rsk164ejJnEVEEiJ40gPliavx2VVV7Y_Ct4f2WoFIJsBt9HXF1wHcu3KMKtTXlRuPq9VD2Mm4WZxwnLNv9A5W-N_ChM2JRr31VG0h9QwEsB7o0wOITiEyDsWDsi-zN0MBeG9OQVFt9avvd1sLp4O_PGsajiVWRoW4BMbi3jKEGEZs4GbDZUWLgEn5gXcYx8Wk8fBf0VoaYYRLjR9BT60n7zDOWNr_vwKlc7oqCSbwHyehXT-ToXIv-O-qQxj34350XXQV1NATw5k2oqe9ym48pEAmPxPKjXZ2UfIfDFZbrmg0vu9ryhabIdrjzvVVaboCWOuQ9GsZA5gKIzGGpviKyETc7Q9wfIsvy670BsF5YsmjkROTamViHMYYqGtLmEq_2HXNXrFiB-t1gu5Ssj87ilz1MgyRcvEZywuUlrpKns5Nb-R1SShU8JnmvWaKSLWq1Hhtnoa6p3HzJdp2hEPPVGlLAsptPVPwGSvGUzl3R_5q1xNv2FLQ94Cud9hsV0hAuY5b_qQRQmMape8h_0TcKNNYyF7j_H5GkF7XovV7nJswMDqHIm7Ez_YmK2wzaqVSYiu8YioHdXW_6Gvu4ZeKpx_AmPDTYYv8uVTKSfZ3gdqGpXOnjY7uWHC-3beOyjApiFkoOgBVK10CtM16Xg3A17Zbs80Gisb5BTuuvuw5Y2H1hGOfUbKWrVTKkFIl2QlKX"

In [ ]:
dbx = dropbox.Dropbox(ACCESS_TOKEN)

dropbox_path = "/IOE 524 F25 Team #1/Katie_CES_20251018(Data) updated.csv"

_, res = dbx.files_download(path=dropbox_path)
data = pd.read_csv(BytesIO(res.content))

print(data.head())

   ED_ARRIVAL_DTTM      ED_FIRST_CHIEF_COMPLAINT  ED_TRIAGE_ACUITY_LEVEL  \
0   7/29/2024 4:10        Swallowed Foreign Body                       3   
1  5/23/2025 10:31               Facial Swelling                       3   
2  9/11/2024 17:48          Respiratory Distress                       2   
3  9/12/2024 15:55                      Headache                       3   
4   1/7/2025 14:24  Abdominal Pain - Generalized                       3   

   AGE_AT_VISIT_YRS ED_TRIAGE_START_DTTM ED_TRIAGE_STOP_DTTM   ED_ROOMED_DTTM  \
0             6.875       7/29/2024 4:14      7/29/2024 4:16   7/29/2024 4:18   
1             3.600      5/23/2025 10:35     5/23/2025 10:42  5/23/2025 10:34   
2             2.491      9/11/2024 18:10     9/11/2024 18:14  9/11/2024 17:54   
3            12.993      9/12/2024 15:59     9/12/2024 16:07  9/12/2024 16:09   
4            12.896       1/7/2025 14:26      1/7/2025 14:34   1/7/2025 14:29   

  PROVIDER_ASSIGNED_DTTM ED_PROVIDER_FIRST_CONTACT_DTTM 

Data cleaning

In [4]:
cols = ["ED_TRIAGE_ACUITY_LEVEL","TRIAGE_START_TO_END_MIN", "ARRIVAL_TO_TRIAGE_START_MIN"]
data = data[~(data[cols] <= 0).any(axis=1)]
data = data[~(data["AGE_AT_VISIT_YRS"] >= 22)]
print(len(data))

39175


Indices

In [ ]:
# Indices
# T = list(range(1, 145))  # 10 min
T = list(range(1, 45)) # 10 min: 0:00-7:20
L = list(range(1, 6))
D = list(range(1, 6))


nurse_shift_map = {
    1: list(range(43, 67)),                                 # 07:00-11:00
    2: list(range(67, 91)),                                 # 11:00-15:00
    3: list(range(91, 115)),                                # 15:00-19:00
    4: list(range(115, 139)),                               # 19:00-23:00
    5: list(range(139, 145)) + list(range(1, 19)),          # 23:00-03:00
    6: list(range(19, 43))                                  # 03:00-07:00
}

time_map_nurse = {
    t: [u for u in nurse_shift_map if t in nurse_shift_map[u]]
    for t in T
}

Patient arrivals P[t, l, d] from data 

In [6]:
data = data[data['ARRIVAL_TO_LEAVE_CES_HOUR']<=24]
print(len(data))
df = data[['ED_ARRIVAL_DTTM', 'ED_TRIAGE_ACUITY_LEVEL', 'ARRIVAL_TO_LEAVE_CES_HOUR', 'TRIAGE_START_TO_END_MIN', "ED_DISPOSITION_ENTRY_DTTM", "ED_PROVIDER_FIRST_CONTACT_DTTM"]].copy()

df = df.dropna(subset=['TRIAGE_START_TO_END_MIN'])

df['ED_ARRIVAL_DTTM'] = pd.to_datetime(data['ED_ARRIVAL_DTTM'], format='%m/%d/%Y %H:%M')
df['ED_TRIAGE_ACUITY_LEVEL'] = df['ED_TRIAGE_ACUITY_LEVEL'].astype(int)
df['ARRIVAL_TO_LEAVE_CES_HOUR'] = pd.to_numeric(data['ARRIVAL_TO_LEAVE_CES_HOUR'], errors='coerce')
df['TRIAGE_START_TO_END_MIN'] = pd.to_numeric(data['TRIAGE_START_TO_END_MIN'], errors='coerce')
df['ARRIVAL_TO_LEAVE_CES_MIN'] = pd.to_numeric(data['ARRIVAL_TO_LEAVE_CES_HOUR'], errors='coerce')*60
df["ED_DISPOSITION_ENTRY_DTTM"] = pd.to_datetime(data["ED_DISPOSITION_ENTRY_DTTM"], errors='coerce')
df["ED_PROVIDER_FIRST_CONTACT_DTTM"] = pd.to_datetime(data["ED_PROVIDER_FIRST_CONTACT_DTTM"], errors='coerce')
df["PROVIDER_STAY"] = df["ED_DISPOSITION_ENTRY_DTTM"] - df["ED_PROVIDER_FIRST_CONTACT_DTTM"]
df["PROVIDER_STAY"] = df["PROVIDER_STAY"].dt.total_seconds() / 60

# df = df[~(df["PROVIDER_STAY"] <= 0)]

df['l'] = df['ED_TRIAGE_ACUITY_LEVEL'].astype(int)

df['t'] = df['ED_ARRIVAL_DTTM'].dt.hour + 1
df['arrival_date'] = df['ED_ARRIVAL_DTTM'].dt.date


duration_min = df['ARRIVAL_TO_LEAVE_CES_MIN']
def get_stay_category(duration_hours):
    if duration_hours < 120:
        return 1
    elif duration_hours < 360:
        return 2
    elif duration_hours < 720:
        return 3
    else:
        return 4
        
df['d'] = df['ARRIVAL_TO_LEAVE_CES_MIN'].apply(get_stay_category)
df.columns = df.columns.str.replace(r'\.\.\.', '', regex=True).str.strip()


df['t'] = (df['ED_ARRIVAL_DTTM'].dt.hour * 60 + df['ED_ARRIVAL_DTTM'].dt.minute) // 10 + 1


arrival_counts = df.groupby(['arrival_date', 't', 'l', 'd']).size().reset_index(name='P_count')

daily_totals = arrival_counts.groupby('arrival_date').size()
busiest_day = daily_totals.idxmax()
median_arrivals = daily_totals.median()

normal_day = (daily_totals - median_arrivals).abs().idxmin()

busiest_day_P_df = arrival_counts[arrival_counts['arrival_date'] == busiest_day].copy()
normal_day_P_df = arrival_counts[arrival_counts['arrival_date'] == normal_day].copy()


def generate_p_dict(df_p_data, T, L, D):
    P = {}
    
    for t in T:
        for l in L:
            for d in D:
                P[t, l, d] = 0
                
    for index, row in df_p_data.iterrows():
        t = int(row['t']) # t'
        l = int(row['l'])
        d = int(row['d'])
        value = int(row['P_count'])
        
        if t in T and l in L and d in D:
            P[t, l, d] = value
            
    return P

P_busiest = generate_p_dict(busiest_day_P_df, T, L, D)
P_normal = generate_p_dict(normal_day_P_df, T, L, D)



print(f"Busiest Day: {busiest_day}, total arrivals: {daily_totals.max():.0f}")
print(f"Normal Day: {normal_day}, total arrivals: {daily_totals[normal_day]:.0f} (median: {median_arrivals:.2f})")


37282
Busiest Day: 2025-02-10, total arrivals: 127
Normal Day: 2024-09-02, total arrivals: 88 (median: 88.00)


Calculate triage time m[l, d] & treatment time M[l, d]

In [8]:
df['TRIAGE_START_TO_END_MIN'] = pd.to_numeric(data['TRIAGE_START_TO_END_MIN'], errors='coerce')
df['ARRIVAL_TO_LEAVE_CES_MIN'] = pd.to_numeric(data['ARRIVAL_TO_LEAVE_CES_HOUR'], errors='coerce')*60
df['PROVIDER_STAY'] = pd.to_numeric(df['PROVIDER_STAY'], errors='coerce')

m = {} 
M = {}


for l in L:
    for d in D:
        sub = df[(df['l'] == l) & (df['d'] == d)]

        triage_series = sub['TRIAGE_START_TO_END_MIN'].dropna()
        provider_series = sub['PROVIDER_STAY'].dropna()

        if len(triage_series) == 0:
            fallback_triage = df[df['l']==l]['TRIAGE_START_TO_END_MIN'].median()
        else:
            fallback_triage = triage_series.mean()

        if len(provider_series) == 0:
            fallback_provider = df[df['l']==l]['PROVIDER_STAY'].median()
        else:
            fallback_provider = provider_series.mean()

        m[(l,d)] = max(1, int(round(float(fallback_triage) / 10.0)))
        M[(l,d)] = max(1, int(round(float(fallback_provider) / 10.0)))



In [ ]:
service_rooms = {'triage':['tri1','tri2','tri3','tri4'],
                 'r1':['16','1','2'],
                 'r2':['3','4','5'],
                 'r3': ['6','7','17'],
                 'r4':['18','19','21'],
                 'r5':['W1','W2','W3','W4'],
                 'r6':['W5','W6','W7','W8'],
                 'r7':['8','9','10'],
                 'r8':['11','12','28'],
                 'r9':['13','14','15'],
                 'r10':['H6','H7','H8','H9'],
                 'r11':['W9','Q1','Q2','X1'],
                 'r12':['22','23','24'],
                 'r13':['25','26','27'],
                 'r14':['20','29','31'],
                 'r15':['30','H1','H2'],
                 'r16':['H3','H4','H5'],
                 'r17':['38','39'],
                 'r18':['40','41'],
                 'r19':['32','33','34'],
                 'r20':['35','36','37']}

queues = {'q_triage':['tri1','tri2','tri3','tri4'],
          'q_r1':['16','1','2'],
          'q_r2':['3','4','5'],
          'q_r3': ['6','7','17'],
          'q_r4':['18','19','21'],
          'q_r5':['W1','W2','W3','W4'],
          'q_r6':['W5','W6','W7','W8'],
          'q_r7':['8','9','10'],
          'q_r8':['11','12','28'],
          'q_r9':['13','14','15'],
          'q_r10':['H6','H7','H8','H9'],
          'q_r11':['W9','Q1','Q2','X1'],
          'q_r12':['22','23','24'],
          'q_r13':['25','26','27'],
          'q_r14':['20','29','31'],
          'q_r15':['30','H1','H2'],
          'q_r16':['H3','H4','H5'],
          'q_r17':['38','39'],
          'q_r18':['40','41'],
          'q_r19':['32','33','34'],
          'q_r20':['35','36','37']}

dummy = ['Out']

spaces_route = {}

spaces = dummy + list(chain(*list(service_rooms.values()))) + list(queues.keys())

for tri in service_rooms['triage']:
    spaces_route[tri] = [tri]
    for r in service_rooms:
        if 'tri' not in r:
            spaces_route[tri] += service_rooms[r] + [f'q_{r}']

for q in queues:
    spaces_route[q] = [q]+queues[q]

for r in service_rooms:
    if 'tri' not in r:
        for r_i in service_rooms[r]:
            spaces_route[r_i] = [r_i]+['Out']

spaces_route['Out'] = ['Out']



Parameters

In [ ]:
P = P_busiest
# P = P_normal

# w_l
w = {}
w_values = [8.0, 4.0, 2.0, 1.0, 0.5]
for l in L:
    w[l] = w_values[l - 1]

treat_room = [
    r_i
    for r in service_rooms
    if r != 'triage'
    for r_i in service_rooms[r]
]

J = int(np.ceil(len(treat_room) / 2))

In [11]:
# Track patient i (get his/her acuity, duration, arrival time)
I = []
l_of, d_of, a_of, end_of = {}, {}, {}, {}
idx = 0
for t in T:
    for l in L:
        for d in D:
            n = P.get((t,l,d), 0)
            for _ in range(n):
                idx += 1
                I.append(idx)
                l_of[idx] = l
                d_of[idx] = d
                a_of[idx] = t


In [ ]:
model = gb.Model('MILP_Staffing')

staff,patient,provider = {},{},{} # Main variables
room_service,patient_insystem = {},{} # Auxiliary variables

# Triage service indicator: patient + triage nurse
served_triage = {}

# Room service indicator: patient + nurse + provider
served_room = {}

# Triage time shortage for each patient 
shortfall_triage = {}

# Room time shortage for each patient
shortfall_room = {}

for u in nurse_shift_map:
    for r_set in service_rooms:
        if r_set != 'triage':
            staff[u,r_set] = model.addVar(name=f'staff_{u}_{r_set}',vtype=GRB.BINARY)
    for tri_room in service_rooms['triage']:
        staff[u,tri_room] = model.addVar(name=f'staff_{u}_{tri_room}',vtype=GRB.BINARY)




for i in I:
    patient_insystem[i] = model.addVar(name=f'patient_{i}_in_system',vtype=GRB.BINARY)
    for r in spaces:
        for t in range(a_of[i],len(T)+1): # Only initializing their variables when they arrive
            patient[i,r,t] = model.addVar(name=f'patient_{i}_{r}_{t}',vtype=GRB.BINARY)


# Computing the max number of providers given the total number of rooms
J = int(np.ceil(len(service_rooms['r1']+service_rooms['r2'])/2))

for j in range(J):
    for r in service_rooms:
        if r != 'triage':
            for r_i in service_rooms[r]:
                for t in T:
                    provider[j,r_i,t] = model.addVar(name=f'provider_{j}_{r}_{t}',vtype=GRB.BINARY)


for r in service_rooms:
    if r != 'triage':
        for r_i in service_rooms[r]:
            for t in T:
                room_service[r_i,t] = model.addVar(name=f'room_serving_{r_i}_{t}',vtype=GRB.BINARY)


for i in I:
    for t in range(a_of[i],T[-1]+1): #Only initializing their variables when they arrive
        
        if t == a_of[i]:
            model.addConstr(patient[i,'q_triage',t]+ gb.quicksum(patient[i,r,t] for r in service_rooms['triage']) == 1, name=f"Arrive_{i}")
        
        model.addConstr(
            (gb.quicksum(patient[i,r,t] for r in spaces) == 1),
            name=f"One_state_per_patient_per_time_{i}_{t}"
        )

        if t < T[-1]:
            for r in spaces:
                model.addConstr(patient[i,r,t] <= gb.quicksum(patient[i,r2,t+1] for r2 in spaces_route[r]))

    model.addConstr(
            patient[i,'Out',T[-1]] + patient_insystem[i] == 1,
            name=f"Disjunction_Out_In_{i}"
        )


for t in T:
    for j in range(J):
        model.addConstr(gb.quicksum(provider[j,r_i,t] 
                                    for r in service_rooms 
                                    for r_i in service_rooms[r] 
                                    if r != 'triage') <= 2,name='At most 2 rooms at any given time' )
    
    for r in service_rooms:
        for r_i in service_rooms[r]:

            model.addConstr(gb.quicksum(patient[i,r_i,t] for i in I if t >= a_of[i]) <= 1,name=f'Only one patient per service room {r_i} at {t}')

            if r == 'triage':
                model.addConstr(
                    gb.quicksum(patient[i, r_i, t] for i in I if t >= a_of[i]) <=
                    gb.quicksum(staff[u, r_i] for u in time_map_nurse[t]),
                    name=f'Cap_triage_room_{r_i}_t{t}'
                )
            
            else:
                model.addConstr(gb.quicksum(patient[i,r_i,t] for i in I if t >= a_of[i]) <= 
                                gb.quicksum(staff[u,r] for u in time_map_nurse[t]), name=f"Cap_room_{t}_{r}")

            if r != 'triage':
                model.addConstr(room_service[r_i,t] <= gb.quicksum(provider[j,r_i,t] for j in range(J)),name=f'Providers_make_room_{r_i}_serve')

                model.addConstr(room_service[r_i,t] <= gb.quicksum(staff[u,r] for u in time_map_nurse[t]),name=f'Nurses_make_room_{r_i}_serve')
    


# Triage service time
for i in I:
    for tri_room in service_rooms['triage']:
        for t in range(a_of[i], T[-1] + 1):
            served_triage[i, tri_room, t] = model.addVar(
                vtype=GRB.BINARY,
                name=f"servedT_{i}_{tri_room}_{t}"
            )

            model.addConstr(
                served_triage[i, tri_room, t] <= patient[i, tri_room, t],
                name=f"servedT_le_patient_{i}_{tri_room}_{t}"
            )

            model.addConstr(
                served_triage[i, tri_room, t] <= 
                gb.quicksum(staff[u, tri_room] for u in time_map_nurse[t]),
                name=f"servedT_le_nurse_{i}_{tri_room}_{t}"
            )

            model.addConstr(
                served_triage[i, tri_room, t] >=
                patient[i, tri_room, t] +
                gb.quicksum(staff[u, tri_room] for u in time_map_nurse[t]) - 1,
                name=f"servedT_ge_both_{i}_{tri_room}_{t}"
            )


    shortfall_triage[i] = model.addVar(
        vtype=GRB.CONTINUOUS,
        lb=0.0,
        name=f"shortfall_triage_{i}"
    )

    triage_served_i = gb.quicksum(
        served_triage[i, tri_room, t]
        for tri_room in service_rooms['triage']
        for t in range(a_of[i], T[-1] + 1)
    )

    model.addConstr(
        triage_served_i + shortfall_triage[i] == m[l_of[i], d_of[i]],
        name=f"TriageTime_balance_{i}"
    )

    model.addConstr(
        shortfall_triage[i] <= m[l_of[i], d_of[i]] * patient_insystem[i],
        name=f"Incomplete_triage_stay_in_system_{i}"
    )


# Room service time
for i in I:
    for r in service_rooms:
        if r != 'triage': 
            for r_i in service_rooms[r]:
                for t in range(a_of[i], T[-1] + 1):
                    served_room[i, r_i, t] = model.addVar(
                        vtype=GRB.BINARY,
                        name=f"servedR_{i}_{r_i}_{t}"
                    )


                    model.addConstr(
                        served_room[i, r_i, t] <= patient[i, r_i, t],
                        name=f"servedR_le_patient_{i}_{r_i}_{t}"
                    )

                    model.addConstr(
                        served_room[i, r_i, t] <= room_service[r_i, t],
                        name=f"servedR_le_roomservice_{i}_{r_i}_{t}"
                    )

                    model.addConstr(
                        served_room[i, r_i, t] >=
                        patient[i, r_i, t] + room_service[r_i, t] - 1,
                        name=f"servedR_ge_both_{i}_{r_i}_{t}"
                    )


    shortfall_room[i] = model.addVar(
        vtype=GRB.CONTINUOUS, lb=0.0,
        name=f"shortfall_room_{i}"
    )

    treat_served_i = gb.quicksum(
        served_room[i, r_i, t]
        for r in service_rooms if r != 'triage'
        for r_i in service_rooms[r]
        for t in range(a_of[i], T[-1] + 1)
    )

    model.addConstr(
        treat_served_i + shortfall_room[i] == M[l_of[i], d_of[i]],
        name=f"TreatTime_balance_{i}"
    )

    model.addConstr(
        shortfall_room[i] <= M[l_of[i], d_of[i]] * patient_insystem[i],
        name=f"Incomplete_treat_stay_in_system_{i}"
    )



# Patient can go to room only if triage is finished
for i in I:
    for t in range(a_of[i], T[-1] + 1):

        cum_triage_served = gb.quicksum(
            served_triage[i, tri_room, tau]
            for tri_room in service_rooms['triage']
            for tau in range(a_of[i], t + 1)
        )

        for r in service_rooms:
            if r == 'triage':
                continue
            for r_i in service_rooms[r]:
                model.addConstr(
                    cum_triage_served >= m[l_of[i], d_of[i]] * patient[i, r_i, t],
                    name=f"Room_after_full_triage_{i}_{r_i}_{t}"
                )


# Objective function
total_weighted_wait = gb.quicksum(
    w[l_of[i]] * 
        patient[i, r_i, t]
    for i in I for t in range(a_of[i],T[-1]+1) for r_i in spaces if r_i != 'Out'
)

total_staff = gb.quicksum(staff[u, r] for u in nurse_shift_map for r in service_rooms if r != 'triage') + \
    gb.quicksum(staff[u, tri_room] for u in nurse_shift_map for tri_room in service_rooms['triage'])

model.Params.MIPFocus = 3   # emphasize bound improvement
model.Params.Cuts     = 3   # more aggressive cuts
model.Params.Presolve = 2   # keep presolve strong
model.Params.Heuristics = 0.05  # less time hunting for more incumbents
model.Params.MIPGap    = 0.25   # stop at 5% gap        

obj = total_weighted_wait + gb.quicksum(patient_insystem[i] for i in I) - gb.quicksum(patient[i,'Out',T[-1]] for i in I)


obj1 = (
    gb.quicksum(shortfall_room[i]   for i in I)
  + gb.quicksum(shortfall_triage[i] for i in I)
)

model.setObjective(obj1, GRB.MINIMIZE)
model.update()
model.optimize()


total_shortfall_best = (
    sum(shortfall_room[i].X   for i in I)
  + sum(shortfall_triage[i].X for i in I)
)
print("Best total shortfall =", total_shortfall_best)



eps = 1e-6  
model.addConstr(
    gb.quicksum(shortfall_room[i]   for i in I)
  + gb.quicksum(shortfall_triage[i] for i in I)
  <= total_shortfall_best + eps,
  name="Fix_total_shortfall_to_optimum"
)


obj2 = (
    total_weighted_wait
    + gb.quicksum(shortfall_room[i]   for i in I)
    + gb.quicksum(shortfall_triage[i] for i in I)
    + gb.quicksum(patient_insystem[i] for i in I)
    - gb.quicksum(patient[i, 'Out', T[-1]] for i in I)
    + total_staff
)

model.setObjective(obj2, GRB.MINIMIZE)


model.Params.MIPGap    = 0.34   # stop at 5% gap   
model.update()
model.optimize()


Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-01
Set parameter MIPFocus to value 3
Set parameter Cuts to value 3
Set parameter Presolve to value 2
Set parameter Heuristics to value 0.05
Set parameter MIPGap to value 0.25
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  0.25
MIPFocus  3
Cuts  3
Presolve  2

Optimize a model with 92160 rows, 47125 columns and 1287823 nonzeros
Model fingerprint: 0xb17fdf22
Variable types: 22 continuous, 47103 integer (47103 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 243.0000000
Presolve removed 15651 rows and 4901 columns (presolve time = 6s)...
Presolve removed 15651 rows and 4901 columns (pre

In [ ]:
for s in nurse_shift_map:
    triage_rooms_with_nurse = [
        tri_room for tri_room in service_rooms['triage']
        if staff[s, tri_room].X == 1
    ]
    print(f"Shift {s}: {len(triage_rooms_with_nurse)} triage nurses ")

    for r_set in service_rooms:
        if r_set == 'triage':
            continue
        if staff[s, r_set].X == 1:
            print(f"  - 1 nurse on room set {r_set}")


for j in range(J):
    prov_room = []
    for r in service_rooms:
        if r != 'triage':
            for r_i in service_rooms[r]:
                for t in T:
                    if provider[j,r_i,t].x == 1:
                        prov_room.append(r_i)
    print(prov_room)


print('--------------------------------------------------------------')

paths = {}

for i in I:
    paths[i]=[]
    for t in T:
        if t < a_of[i]:
            paths[i].append('Pre')
        else:
            for r in spaces:
                if patient[i,r,t].x == 1:
                    paths[i].append(r)
    print(paths[i])

Shift 1: 0 triage nurses 
  - 1 nurse on room set r1
  - 1 nurse on room set r3
  - 1 nurse on room set r10
  - 1 nurse on room set r11
Shift 2: 0 triage nurses 
Shift 3: 0 triage nurses 
Shift 4: 0 triage nurses 
Shift 5: 2 triage nurses 
  - 1 nurse on room set r1
  - 1 nurse on room set r3
  - 1 nurse on room set r14
Shift 6: 1 triage nurses 
  - 1 nurse on room set r1
  - 1 nurse on room set r3
  - 1 nurse on room set r10
  - 1 nurse on room set r11
  - 1 nurse on room set r14
['16', '16', '16', '16', '16', '2', '2', '2', '5', '5', '5', '6', '7', '7', '7', '7', '17', '17', '17', '17', '17', 'W6', 'W6', 'H6', 'H6', 'H7', 'H7', 'H7', 'H7', 'W9', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q2', 'Q2', '20', '20', '20', '20', '20', '20', '20', '20']
['16', '16', '16', '16', '16', '16', '2', '2', '2', '2', '2', '2', '5', '6', '7', '7', '17', '17', '17', '17', '17', 'W7', 'H6', 'H8', 'H8', 'H8', 'H8', 'H8', 'W9', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q2', 'Q2', 'X1', 'X1', 'X1',